<a href="https://colab.research.google.com/github/MilanCugur/Offline_Writer_Identification/blob/master/Baseline_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up ImageDisk and InfoDisk

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import numpy as np
from zipfile import ZipFile

from keras.utils import Sequence
from random import shuffle

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

Using TensorFlow backend.


In [0]:
%%time
# Extract images to 'ImageDisk' folder

path = './drive/My Drive/New_Approach_29.12.2018/ImageDisk.zip'  # Different .zip folder with images
archive = ZipFile(path, 'r')
archive.extractall('./ImageDisk')  
archive.close()

CPU times: user 1min 23s, sys: 39.7 s, total: 2min 3s
Wall time: 2min 13s


In [0]:
# Extract info about the images to InfoDisk list (path, ascii, writer_id); 
# example: ('f0500_38/l0500_38/l0500_38_00008.png', '106', '500')
path = './drive/My Drive/New_Approach_29.12.2018/InfoDisk.txt'  
InfoDisk = []

with open(path, "r") as f:
  for line in f.readlines():
    line = line.rstrip('\n').split(' ')
    InfoDisk.append((line[0], line[1], line[2]))
f.close()

# OneHot

In [0]:
# OneHot Encoding
class OneHot():  
  # classes = list of classes : 0-9, a-z, A-Z
  # n = number of classes     : 62
  
  def __init__(self, classes):
    self.classes = classes
    self.n = len(classes)
    
  def encode(self, class_name):
    one_hot = np.zeros(shape=(self.n), dtype=np.int8)
    class_index = self.classes.index(class_name)
    one_hot[class_index] = 1
    return one_hot
  
  def encode_all(self, list_class_names):
    return np.array([self.encode(class_name) for class_name in list_class_names])
  
  def decode(self, one_hot):
    class_index = one_hot.argmax()
    return self.classes[class_index]
  
  def decode_all(self, list_one_hots):
    return np.array([self.decode(one_hot) for one_hot in list_one_hots])

In [0]:
labels = set()
for (path, label, writer) in InfoDisk:
  labels.add(label)

print(sorted(labels))  # 62 ascii codes in string format
print(len(labels))

['100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '97', '98', '99']
62


In [0]:
OH_L = OneHot(sorted(labels))

# Model

In [0]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPool2D, Flatten, Concatenate, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam  # First, naive approach
from keras.losses import categorical_crossentropy
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger  # Add callbacks to model
from keras.models import load_model  # Save model params
from keras.layers import LeakyReLU, concatenate
from keras.layers.advanced_activations import PReLU
from keras.initializers import glorot_normal

import keras.backend as K

In [0]:
BOX_SIZE = 28                          # (28, 28) images; fixed size; I already preprocessed it
image_shape = (BOX_SIZE, BOX_SIZE, 1)  # (28, 28, 1) shape for tensorflow
number_of_classes = len(labels)        # 62 classes; 0-9, a-z, A-Z

In [0]:
x_input = Input(shape=image_shape)  # shape=(?, 24, 24, 1)

x = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation='relu', padding='valid')(x_input)  # shape=(?, 62, 62, 32)
x = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), activation='relu', padding='valid')(x) 
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2,2), padding='same')(x)                                                      # shape=(?, 31, 31, 32)
x = Conv2D(filters=48, kernel_size=(3,3), strides=(1,1), activation='relu', padding='valid')(x)        # shape=(?, 29, 29, 32)
x = Conv2D(filters=48, kernel_size=(3,3), strides=(1,1), activation='relu', padding='valid')(x) 
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2,2), padding='same')(x)                                                      # shape=(?, 15, 15, 32)
x = Conv2D(filters=64, kernel_size=(2,2), strides=(1,1), activation='relu', padding='valid')(x)        # shape=(?, 14, 14, 32)
x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu', padding='valid')(x) 
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(3, 3), padding='same')(x)                                                     # shape=(?, 5, 5, 32); dim: 800; with (2,2) dim is ~1500
x = Flatten()(x)                                                                                       # shape=(?, ?); depends on mini btch size; alternative batch_flatten()

x = Dense(units=768, activation='relu')(x)        # shape=(?, 768) 
x = Dropout(rate=0.25)(x)                          # shape=(?, 768) 
x = Dense(units=256, name='next_to_last')(x)                           # shape=(?, 256)
x = Dropout(rate=0.25)(x)                          # shape=(?, 256)

y_output = Dense(units=number_of_classes, activation='softmax')(x)  # shape=(?, 62) 

In [0]:
recognizer = Model(inputs=x_input, outputs=y_output, name='recognizer')

In [0]:
recognizer.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 48)        13872     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 48)          20784     
__________

In [0]:
recognizer.compile(optimizer=Adam(lr=1e-3), loss=categorical_crossentropy, metrics=['accuracy'])   # beta_1=0.9, beta_2=0.999

In [0]:
reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, verbose=1, mode='min', #add min_delta
                           cooldown=1, min_le=1e-8)

estop = EarlyStopping(monitor='val_loss', min_delta=0, patience=4, verbose=1, mode='min')

excel = CSVLogger(filename='./baseline_hacked.csv', separator=',', append=False)

checkp = ModelCheckpoint('./best_w.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)

# Data

In [0]:
writers = {}
for path, label, writer in InfoDisk:
  writer = int(writer)
  if writer in writers:
    writers[writer] += 1
  else:
    writers[writer] = 1
  
writers = list(writers.items())
writers = sorted(writers, key=lambda x: x[1])
print(writers)

[(1727, 18), (825, 21), (1725, 24), (1965, 30), (2027, 33), (1726, 39), (2046, 40), (2044, 41), (1995, 43), (1990, 46), (1992, 46), (1984, 49), (1974, 49), (1767, 50), (1723, 51), (1969, 51), (1556, 67), (1756, 67), (2067, 72), (1110, 76), (4043, 77), (1977, 79), (1209, 79), (1749, 86), (1006, 86), (1731, 87), (2504, 90), (2097, 91), (1598, 92), (3704, 92), (3901, 94), (1931, 95), (416, 95), (1348, 95), (1831, 97), (3642, 98), (1016, 98), (1280, 99), (1854, 99), (1675, 100), (1923, 100), (3696, 102), (1621, 102), (1733, 102), (2105, 102), (1576, 103), (1432, 103), (1783, 103), (3797, 103), (1542, 103), (2048, 104), (1607, 105), (2095, 105), (785, 105), (2069, 106), (1919, 106), (1646, 107), (2047, 107), (2077, 107), (1517, 107), (1523, 107), (1386, 107), (4064, 107), (1859, 107), (2040, 107), (1548, 108), (3240, 108), (3820, 108), (3747, 108), (1802, 108), (1944, 109), (1639, 109), (2032, 109), (1788, 110), (1671, 111), (1686, 112), (3296, 112), (2099, 112), (2080, 113), (1655, 113), (

In [0]:
testW = []
countImgs = 0
while countImgs<len(InfoDisk)*0.1:
  testW.append(writers[-1])
  countImgs += writers[-1][1]
  writers = writers[:-1]

In [0]:
print(testW)
len(testW)

[(261, 583), (289, 464), (27, 446), (292, 442), (155, 442), (119, 437), (10, 436), (644, 436), (1301, 435), (337, 435), (199, 433), (70, 433), (164, 430), (46, 430), (254, 429), (190, 428), (66, 428), (54, 428), (344, 427), (335, 427), (203, 426), (320, 426), (110, 426), (743, 425), (315, 425), (11, 425), (281, 424), (17, 424), (246, 423), (99, 423), (1369, 422), (395, 422), (256, 422), (863, 422), (115, 421), (465, 420), (360, 420), (317, 420), (495, 419), (451, 419), (996, 419), (23, 418), (379, 418), (255, 418), (8, 418), (443, 417), (330, 417), (312, 416), (197, 416), (102, 416), (35, 416), (613, 416), (161, 415), (361, 415), (698, 415), (679, 415), (547, 415), (473, 414), (843, 414), (187, 414), (108, 414), (63, 413), (1, 413), (925, 413), (633, 413), (208, 413), (534, 413), (472, 412), (494, 412), (347, 412), (1091, 411), (172, 411), (328, 411), (94, 410), (622, 410), (77, 409), (118, 408), (211, 408), (804, 408), (530, 408), (401, 407), (31, 407), (815, 407), (752, 407), (690, 4

200

In [0]:
# Split and stratify
# 80% : 10% : 10%
# shuffle(writers)
trainW = []
validationW = []

for i in range(0, len(writers)):
  if i%10==9:
    validationW.append(writers[i])
  else:
    trainW.append(writers[i])

trainW = set(trainW)
validationW = set(validationW)
testW = set(testW)

In [0]:
testW = [x[0] for x in testW]
validationW = [x[0] for x in validationW]
trainW = [x[0] for x in trainW]

In [0]:
print(len(trainW))
print(len(validationW))
print(len(testW))

3057
339
200


In [0]:
%%time
train_Images = []
train_Labels = []
train_Writers = []

validation_Images = []
validation_Labels = []
validation_Writers = []

test_Images = []
test_Labels = []
test_Writers = []

shuffle(InfoDisk)  # for randomization of labels; writers are fitted anywhay
for path, label, writer in InfoDisk:
  writer = int(writer)
  img = mpimg.imread(os.path.join('./ImageDisk', path))
  if writer in testW:
    test_Images.append(img.reshape(28, 28, 1)/255.0)
    test_Labels.append(OH_L.encode(label))
    test_Writers.append(writer)
  elif writer in validationW:
    validation_Images.append(img.reshape(28, 28, 1)/255.0)
    validation_Labels.append(OH_L.encode(label))
    validation_Writers.append(writer)
  elif writer in trainW:
    train_Images.append(img.reshape(28, 28, 1)/255.0)
    train_Labels.append(OH_L.encode(label))
    train_Writers.append(writer)
  else:
    print('Unrecognized writer, ', writer)  # Throw exception

CPU times: user 2min 11s, sys: 39.6 s, total: 2min 51s
Wall time: 6min 46s


In [0]:
print(len(train_Writers))
print(len(validation_Writers))
print(len(test_Writers))
print()
print(len(train_Labels))
print(len(validation_Labels))
print(len(test_Labels))
print()
print(len(train_Images))
print(len(validation_Images))
print(len(test_Images))

659541
73209
81505

659541
73209
81505

659541
73209
81505


In [0]:
history = recognizer.fit(x=np.array(train_Images), y=np.array(train_Labels),
                         batch_size=1024, epochs=20, verbose=1, 
                         callbacks=[reduce, estop, excel, checkp], 
                         validation_data=(np.array(validation_Images), np.array(validation_Labels)),          
                         shuffle=True)

In [0]:
recognizer.evaluate(x=np.array(test_Images), y=np.array(test_Labels), batch_size=256, verbose=1)

In [0]:
whistory = recognizer.fit(x=np.array(train_Images), y=np.array(train_Labels),
                         batch_size=2048, epochs=10, verbose=1, 
                         callbacks=[reduce, estop, excel, checkp], 
                         validation_data=(np.array(validation_Images), np.array(validation_Labels)),          
                         shuffle=True)

In [0]:
recognizer.evaluate(x=np.array(test_Images), y=np.array(test_Labels), batch_size=256, verbose=1)

In [0]:
history = recognizer.fit(x=np.array(train_Images), y=np.array(train_Labels),
                         batch_size=4096, epochs=5, verbose=1, 
                         callbacks=[reduce, estop, excel, checkp], 
                         validation_data=(np.array(validation_Images), np.array(validation_Labels)),          
                         shuffle=True)

In [0]:
recognizer.evaluate(x=np.array(test_Images), y=np.array(test_Labels), batch_size=256, verbose=1)

## Evaluate fixed weight

In [0]:
recognizer.evaluate(x=np.array(train_Images), y=np.array(train_Labels), batch_size=256, verbose=1)

In [0]:
recognizer.evaluate(x=np.array(validation_Images), y=np.array(validation_Labels), batch_size=256, verbose=1)

In [0]:
recognizer.evaluate(x=np.array(test_Images), y=np.array(test_Labels), batch_size=256, verbose=1)

In [0]:
from keras.models import load_model
recognizer.save_weights('end_training_weights.h5') 

In [0]:
recognizer.load_weights('best_w.h5')

In [0]:
recognizer.evaluate(x=np.array(train_Images), y=np.array(train_Labels), batch_size=256, verbose=1)

In [0]:
recognizer.evaluate(x=np.array(validation_Images), y=np.array(validation_Labels), batch_size=256, verbose=1)

In [0]:
recognizer.evaluate(x=np.array(test_Images), y=np.array(test_Labels), batch_size=256, verbose=1)

In [0]:
recognizer.save_weaights('baseline_weights.h5') 
recognizer.save('baseline_model.h5') 

In [0]:
!cp './baseline_weights.h5' './drive/My Drive/New_Approach_29.12.2018/baseline_weights.h5'
!cp './baseline_model.h5' './drive/My Drive/New_Approach_29.12.2018/baseline_model.h5'

In [0]:
from keras.models import load_model
r = load_model('./drive/My Drive/New_Approach_29.12.2018/baseline_model.h5')
r.evaluate(x=np.array(test_Images), y=np.array(test_Labels), batch_size=256, verbose=1)

81505/81505 [==============================] - 3s 42us/step


[0.3596773250140586, 0.8735046929541149]

# Finder

In [0]:
recognizer.input

<tf.Tensor 'input_1:0' shape=(?, 28, 28, 1) dtype=float32>

In [0]:
recognizer.get_layer('next_to_last').output

<tf.Tensor 'next_to_last/BiasAdd:0' shape=(?, 256) dtype=float32>

In [0]:
finder = Model(inputs=recognizer.input, outputs=recognizer.get_layer('next_to_last').output)

In [0]:
finder.save_weights('finder_weights.h5') 
finder.save('finder_model.h5') 

In [0]:
!cp './finder_weights.h5' './drive/My Drive/New_Approach_29.12.2018/finder_weights.h5'
!cp './finder_model.h5' './drive/My Drive/New_Approach_29.12.2018/finder_model.h5'